In [11]:
import mysql.connector
import pandas as pd

con = mysql.connector.connect(
    host="haru-test.cht2hqsb55a9.ap-northeast-2.rds.amazonaws.com",
    user="admin",
    password="wittew-vorzac-0hYzra",
    database="haru_dev_db"
)

rx = pd.read_csv('Rx.csv').dropna().drop_duplicates()
tx = pd.read_csv('Tx.csv').dropna().drop_duplicates()
vaccine1 = pd.read_csv('Vaccine1.csv').dropna().drop_duplicates()
vaccine2 = pd.read_csv('Vaccine2.csv').dropna().drop_duplicates()
supplies = pd.read_csv('Supplies.csv').dropna().drop_duplicates()
admissionhotel = pd.read_csv('AdmissionHotel.csv').dropna().drop_duplicates()

In [2]:
# codes = [
#     rx[['__typename', 'RxCodeGroup.groupName']].dropna().drop_duplicates(),
#     tx[['__typename', 'TxCodeGroup.groupName']].dropna().drop_duplicates(),
#     vaccine2[['__typename', 'VaccineCodeGroup.groupName']].dropna().drop_duplicates(),
#     supplies[['__typename', 'SuppliesCodeGroup.groupName']].dropna().drop_duplicates(),
#     admissionhotel[['__typename', 'AdmissionHotelCodeGroup.groupName']].dropna().drop_duplicates()
# ]

# for code in codes:
#     for index, row in code.iterrows():
#         cur = con.cursor()
#         sql = """
#             INSERT INTO hospital_pay_category (id, name, sub_name, hospital_id) 
#             VALUES (get_unique_id_from_sequences('hospital_pay_category'), %s, %s, 5877)
#         """
#         cur.execute(sql, (row.values[0], row.values[1]))
#         con.commit()
#         cur.close()

In [3]:
# code = vaccine1[['__typename']].dropna().drop_duplicates()

# for index, row in code.iterrows():
#     cur = con.cursor()
#     sql = f"""
#         INSERT INTO hospital_pay_category (id, name, hospital_id) 
#         VALUES (get_unique_id_from_sequences('hospital_pay_category'), %s, 5877)
#     """
#     cur.execute(sql, (row.values[0],))
#     con.commit()
#     cur.close()

In [8]:
codes = [
    rx[['displayName', '__typename', 'RxCodeGroup.groupName' , 'RxCodePrice.supplyPrice','RxCodePrice.taxItemType']].dropna().drop_duplicates(),
    tx[['displayName', '__typename', 'TxCodeGroup.groupName', 'TxCodePrice.supplyPrice', 'TxCodePrice.taxItemType']].dropna().drop_duplicates(),
    vaccine2[['displayName', '__typename', 'VaccineCodeGroup.groupName', 'VaccineCodePrice.taxAddRule', 'VaccineCodePrice.appearancePrice']].dropna().drop_duplicates(),
    supplies[['displayName', '__typename', 'SuppliesCodeGroup.groupName', 'SuppliesCodePrice.appearancePrice', 'SuppliesCodePrice.taxItemType']].dropna().drop_duplicates(),
    admissionhotel[['displayName', '__typename', 'AdmissionHotelCodeGroup.groupName', 'AdmissionHotelCodePrice.appearancePrice', 'AdmissionHotelCodePrice.taxItemType']].dropna().drop_duplicates()
]

for code in codes:
    for index, row in code.iterrows():
        cur = con.cursor()
        
        sql = """
            INSERT INTO hospital_pay_list (item_id, hospital_id, name, category, price, tax)
            VALUES (
                get_unique_id_from_sequences('hospital_pay_item'),
                5877,
                %s,
                (SELECT id FROM hospital_pay_category WHERE name = %s AND sub_name = %s),
                %s,
                %s
            )
        """
        
        cur.execute(sql, (row.values[0], row.values[1], row.values[2], row.values[3], 1 if row.values[4] == 1 else 0))
        
        con.commit()
        cur.close()

In [12]:
code = vaccine1[['displayName', '__typename', 'VaccineCodePrice.appearancePrice', 'VaccineCodePrice.taxAddRule']].dropna().drop_duplicates()

for index, row in code.iterrows():
    cur = con.cursor()
    
    sql = """
        INSERT INTO hospital_pay_list (item_id, hospital_id, name, category, price, tax)
        VALUES (
            get_unique_id_from_sequences('hospital_pay_item'),
            5877,
            %s,
            (SELECT id FROM hospital_pay_category WHERE name = %s AND sub_name IS NULL),
            %s,
            %s
        )
    """
    
    cur.execute(sql, (row.values[0], row.values[1], row.values[2], 1 if row.values[3] == 1 else 0))
    
    con.commit()
    cur.close()